In [1]:
import numpy as np
import nibabel as nib
import os
import pandas as pd
from tools.common_variables import *
import math

In [2]:
ver_selection = 1
mode = 'betas_split'
version = 'ver' + str(ver_selection)

In [3]:
models_subs = {}
models = models = ["wself", "woself", "oself"]
columns = ["x0", "y0", "sigma", "slope", "intercept",  "var_explained", "mds_ecc", "mds_ang", "roi", "fit_with"]
columns_test = ["x0", "y0", "sigma", "slope", "intercept",  "test_var_explained", "var_explained", "mds_ecc", "mds_ang", "roi", "fit_with"]
columns_mode = columns
if 'split' in mode:
    columns_mode = columns_test
model_study = 'oself'
for sub, n_sessions in valid_subjects_sessions:
    models_subs[sub] = {}
    for m in models:
        m_file = os.path.join(fits_dirs[sub], mode, version, 'models', f'best_fits_{m}_{sub}_{n_sessions}_{mode}_{version}.npy')
        models_subs[sub][m] = pd.DataFrame(np.load(m_file, allow_pickle =True), columns = columns_mode)
        models_subs[sub][m][columns_mode[:-2]] = models_subs[sub][m][columns_mode[:-2]].astype(np.float32)
    sub_m_study = models_subs[sub][model_study]
hier_index = pd.MultiIndex.from_arrays(
    [
        np.tile(list(rois.keys()), len(models_subs)),
        np.repeat(list(models_subs.keys()), len(rois.keys())),
    ],
    names = ["roi", "sub"],
)
models_subs

{'subj02': {'wself':              x0        y0      sigma      slope  intercept  \
  0     -1.050000  0.210308  16.698013  59.369728 -59.207779   
  1     -1.049994  0.176747   2.788363   1.811171  -1.649965   
  2     -1.049991  0.161515   6.771782   9.407562  -9.259690   
  3     -1.049995  0.477887  13.227313  34.493385 -34.323071   
  4     -1.049977  0.253077   3.991856   3.423564  -3.263386   
  ...         ...       ...        ...        ...        ...   
  35273 -0.672387  1.050000   1.029999   0.573064  -0.261257   
  35274 -0.144282  1.050000   0.876113   0.538285  -0.260109   
  35275 -0.001035  1.050000   1.171419   0.767945  -0.463751   
  35276  0.052753  1.050000   1.289275   0.973497  -0.638177   
  35277  0.132124  1.049922   5.492470   9.832393  -9.586050   
  
         test_var_explained  var_explained   mds_ecc   mds_ang roi fit_with  
  0               -0.005170       0.020574  1.064772 -1.373119  V2     LO-2  
  1               -0.004914       0.019918  1.064767 -

In [4]:
models = ['oself', 'woself']
hemis = ['lh', 'rh']
fields = columns_mode

for model in models:
    print(model)
    for (sub, n_sessions) in valid_subjects_sessions:
        print(sub)
        m = models_subs[sub][model]
        for f in fields:
            print(f'\t{f}')
            s = -1
            for hemi in hemis:
                print(f'\t\t{hemi}')
                maskdata_long_file = os.path.join(customrois_dirs[sub], hemi + '.' + sub + '.testrois.mgz')
                maskdata_long = nib.load(maskdata_long_file).get_fdata().squeeze()
                maskdata_file = os.path.join(customrois_dirs[sub], hemi + '.short.reduced.' + sub + '.testrois.npy')
                maskdata = np.load(maskdata_file, allow_pickle = True).astype(int)

                data_out_file = os.path.join(customrois_dirs[sub], 'results_out', hemi + '.' + model + '.' + f + '.mgz')
                if os.path.exists(data_out_file):
                    continue
                data_out = np.zeros(maskdata_long.shape)
                wrongs = []
                for i in range(maskdata_long.shape[0]):
                    voxel = int(maskdata_long[i])
                    if voxel==0 or voxel > 15:
                        continue
                    s += 1
                    if sior_long[voxel] in to_reduce.keys():
                        roi = rois[to_reduce[sior_long[voxel]]]
                    else:
                        roi = rois[sior_long[voxel]]
                    # with this we prove that the order is maintained between the original mgz file
                    # and the final dataframe
                    if roi != rois[m.iloc[s].roi]:     
                        print(f'{i}: {roi} != {rois[m.iloc[s].roi]}')
                        wrongs.append(i)
                    if f in ['roi', 'fit_with']:
                        data_out[i] = rois[m.iloc[s][f]]
                    elif f == 'mds_ang':
                        data_out[i] = math.degrees(m.iloc[s][f]) + 180
                    elif f == 'intercept':
                        data_out[i] = m.iloc[s][f] + abs(m[f].min()) + 1e-08
                    elif f in ['x0', 'y0']:
                        data_out[i] = m.iloc[s][f] + 1.1
                    else:
                        data_out[i] = m.iloc[s][f]
                img = nib.Nifti1Image(np.expand_dims(data_out, axis = (1,2)), np.eye(4))
#                 if f in ['x0', 'y0']:
#                     print(np.mean(data_out))
                nib.loadsave.save(img, data_out_file)
                if f == 'mds_ang':
                    for ang in [45, 90, 135, 180, 225, 270, 305]:
                        data_out_file = os.path.join(customrois_dirs[sub], 'results_out', hemi + '.' + model + '.' + f + '.' + str(ang) + '.mgz')
                        data_out_ang = np.copy(data_out)
                        data_out_ang[data_out != 0] += ang
                        data_out_ang = data_out_ang % 360
                        img = nib.Nifti1Image(np.expand_dims(data_out_ang, axis = (1,2)), np.eye(4))
                        nib.loadsave.save(img, data_out_file)
print('end')

oself
subj02
	x0
		lh
		rh
	y0
		lh
		rh
	sigma
		lh
		rh
	slope
		lh
		rh
	intercept
		lh
		rh
	test_var_explained
		lh
		rh
	var_explained
		lh
		rh
	mds_ecc
		lh
		rh
	mds_ang
		lh
		rh
	roi
		lh
		rh
	fit_with
		lh
		rh
subj03
	x0
		lh
		rh
	y0
		lh
		rh
	sigma
		lh
		rh
	slope
		lh
		rh
	intercept
		lh
		rh
	test_var_explained
		lh
		rh
	var_explained
		lh
		rh
	mds_ecc
		lh
		rh
	mds_ang
		lh


C:\Users\LuisWork\miniconda3\envs\nsddatapaper\lib\site-packages\nibabel\spatialimages.py:495: UserWarning: Using large vector Freesurfer hack; header will not be compatible with SPM or FSL
  hdr.set_data_shape(shape)


		rh
	roi
		lh
		rh
	fit_with
		lh
		rh
subj04
	x0
		lh
		rh
	y0
		lh
		rh
	sigma
		lh
		rh
	slope
		lh
		rh
	intercept
		lh
		rh
	test_var_explained
		lh
		rh
	var_explained
		lh
		rh
	mds_ecc
		lh
		rh
	mds_ang
		lh
		rh
	roi
		lh
		rh
	fit_with
		lh
		rh
subj05
	x0
		lh
		rh
	y0
		lh
		rh
	sigma
		lh
		rh
	slope
		lh
		rh
	intercept
		lh
		rh
	test_var_explained
		lh
		rh
	var_explained
		lh
		rh
	mds_ecc
		lh
		rh
	mds_ang
		lh
		rh
	roi
		lh
		rh
	fit_with
		lh
		rh
subj07
	x0
		lh
		rh
	y0
		lh
		rh
	sigma
		lh
		rh
	slope
		lh
		rh
	intercept
		lh
		rh
	test_var_explained
		lh
		rh
	var_explained
		lh
		rh
	mds_ecc
		lh
		rh
	mds_ang
		lh
		rh
	roi
		lh
		rh
	fit_with
		lh
		rh
woself
subj02
	x0
		lh
		rh
	y0
		lh
		rh
	sigma
		lh
		rh
	slope
		lh
		rh
	intercept
		lh
		rh
	test_var_explained
		lh
		rh
	var_explained
		lh
		rh
	mds_ecc
		lh
		rh
	mds_ang
		lh
		rh
	roi
		lh
		rh
	fit_with
		lh
		rh
subj03
	x0
		lh
		rh
	y0
		lh
		rh
	sigma
		lh
		rh
	slope
		lh
		rh
	intercept
	